# Question Retrieval Index

In [1]:
import os
import constant
import string
import re
import pandas as pd
import numpy as np
from nltk import word_tokenize as lib_tokenizer

from code.bm25 import BM25Gensim
from vncorenlp import VnCoreNLP

In [2]:
k = 2
b = 0.85

vncorenlp_dir = 'resource/vncorenlp/VnCoreNLP-1.2.jar'
annotator = VnCoreNLP(vncorenlp_dir, annotators='wseg')

In [5]:
def batch_annonate(self, text):
    def annotate(annotator):
        def apply(x):
            return ' '.join([' '.join(text) for text in annotator.tokenize(x)])
        return apply
    return list(map(annotate(self.annotator), text))


def strip_context(text):        
    text = text.lower()
    text = text.replace('\n', ' ') 
    text = re.sub(r'\s+', ' ', text) 
    text = text.strip() 
    return text

def post_process(x):
    x = " ".join(word_tokenize(strip_context(x))).strip()
    x = x.replace("\n"," ")
    x = "".join([i for i in x if i not in string.punctuation])
    x = ' '.join([' '.join(text) for text in annotator.tokenize(x)])
    return x

dict_map = dict({})  
def word_tokenize(text): 
    global dict_map 
    words = text.split() 
    words_norm = [] 
    for w in words: 
        if dict_map.get(w, None) is None: 
            dict_map[w] = ' '.join(lib_tokenizer(w)).replace('``', '"').replace("''", '"') 
        words_norm.append(dict_map[w]) 
    return words_norm

In [4]:
df = pd.read_csv('resource/question_retrieval_data/question_retrieval_data.csv')

In [6]:
data = df.question

In [7]:
out_path = 'resource/question_index'

In [8]:
data = list(map(post_process, data))

In [9]:
data

['cho mình hỏi về quy_định cấp bằng tiến_sĩ',
 'quy_định của trường về cấp bằng tiến_sĩ là gì',
 'trường_quy định cấp bằng tiến_sĩ như_thế_nào',
 'làm_sao để được cấp bằng tiến_sĩ',
 'quy_định của trường về luận_án của nghiên_cứu_sinh để được cấp bằng tiến_sĩ là gì',
 'ai thông_qua luận_án của nghiên_cứu_sinh tiến_sĩ',
 'ai là người thông_qua luận_án nghiên_cứu_sinh tiến_sĩ',
 'ai là người xác_nhận nghiên_cứu_sinh đã hoàn_thành việc sửa_đổi bổ_sung luận_án tiến_sĩ',
 'nghiên_cứu_sinh cần sửa_đổi bổ_sung như_thế_nào để được cấp bằng tiến_sĩ',
 'ai xác_nhận nghiên_cứu_sinh đã hoàn_thành việc sửa_đổi bổ_sung luận_án tiến_sĩ',
 'ai là người chịu trách_nhiệm xác_nhận nghiên_cứu_sinh đã hoàn_thành việc sửa_đổi bổ_sung luận_án tiến_sĩ',
 'quyết_định sửa_đổi bổ_sung luận_án tiến_sĩ của ai đưa ra',
 'ai là người ra quyết_định sửa_đổi bổ_sung luận_án tiến_sĩ',
 'nếu cần thẩm_định thì như_thế_nào',
 'thẩm_định luận_án tiến_sĩ theo quy_định nào',
 'kết_quả thẩm_định đạt chuẩn được quy_định ở đâu',

In [9]:
retriever = BM25Gensim(data)
retriever.create_model(out_path, k=k, b=b)

In [14]:
retriever = BM25Gensim()
retriever.load_model(out_path)

In [22]:
question = 'môn thực tập 1 cần biểu mẫu gi?'

In [25]:
question = 'khi nào được hoàn học phí'

In [31]:
question = 'Trình độ anh văn đầu vào và đầu ra của thạc sĩ và tiến sĩ là như thế nào?'

In [34]:
question = 'Cho tôi hỏi về môn thực tập 1'

In [35]:
index, scores = retriever.get_top_result(question)

In [36]:
df.iloc[index]

,question,return,tag
4202,môn thực tập 1 là gì,Môn Thực tập 1 được tổ chức dưới dạng thực hiệ...,faq
4218,cần những biểu mẫu nào cho môn thực tập 1,Khoa KHKTMT sẽ triển khai các biểu mẫu đăng ký...,faq
4207,quy trình đăng ký và triển khai cho môn thực t...,o Học viên đăng ký môn học theo quy trì...,faq
4209,tìm quy trình của psđh cho môn thực tập 1 ở đâu,Môn Thực tập 1 bản chất là một môn học trong k...,faq
4214,thang điểm đánh giá cho thực tập 1,Theo thang đánh giá của môn học thông thường,faq
3181,cho tôi hỏi về khối lượng học tập,QUY ĐỊNH VỀ CẤU TRÚC CHƯƠNG TRÌNH ĐÀO TẠO\n\n\...,policy
4220,môn thực tập 1 và thực tập 2 đăng kí có theo t...,Môn Thực tập 1 và Thực tập 2 không có ràng buộ...,faq
3172,cho tôi hỏi về khối lượng học tập của trường,QUY ĐỊNH VỀ CẤU TRÚC CHƯƠNG TRÌNH ĐÀO TẠO\n\n\...,policy
4238,lương của môn thực tập,Môn thực tập bản chất là một môn học trong CTĐ...,faq
1492,tôi muốn hỏi về cách tính học phí thực tập tốt...,QUY ĐỊNH CHUNG VỀ HỌC VỤ VÀ ĐÀO TẠO\nPHỤ LỤC 4...,policy


In [21]:
scores

array([22.184195, 16.32745 , 13.820196, 13.438559, 12.964383, 12.396642,
       12.208382, 11.927097, 11.777411, 11.421398], dtype=float32)